In [85]:
import pandas as pd
from collections import Counter
import re
import pickle
import numpy as np
import math
from tqdm import tqdm

In [ ]:
!pip inst

In [6]:
DATA_PATH = ''

In [7]:
extra_data = pd.read_csv(DATA_PATH+'extra_data.csv')
punctuations = extra_data['punctuations'].dropna().to_list()
stop_words = extra_data['stop_words'].dropna().to_list()
suffix_u = extra_data['suffix_u'].dropna().to_list()
suffix = extra_data['suffix'].dropna().to_list()
ar = extra_data['ar'].dropna().to_list()
pr = extra_data['pr'].dropna().to_list()
verb_post = extra_data['verb_post'].dropna().to_list()
verb_pre = extra_data['verb_pre'].dropna().to_list()
verbs = extra_data['verbs'].dropna().to_list()
df = pd.read_csv(DATA_PATH+'IR_data.csv')
mokasar = pd.read_csv(DATA_PATH+'mokasar.csv')
df

,id,content,url
0,1,به گزارش ایسنا، پس از استعفا علی نکویی رئیس شا...,https://www.isna.ir/news/99010100077/حواشی-در-...
1,2,به گزارش ایسنا، امیر محسنی با بیان اینکه این ل...,https://www.isna.ir/news/98122922468/ثبت-نام-ب...
2,3,آرش فرهادیان در گفت و گو با ایسنا، درباره آخری...,https://www.isna.ir/news/99010200541/فرهادیان-...
3,4,به گزارش ایسنا، فدراسیون بین المللی شنا قانونی...,https://www.isna.ir/news/99010200528/فناوری-وی...
4,5,به گزارش ایسنا، فدراسیون جهانی صعودهای ورزشی (...,https://www.isna.ir/news/99010200510/تعویق-زما...
...,...,...,...
6995,6996,**به گزارش ایسنا،** سیگار کشیدن، علت عمده‌ برو...,https://www.isna.ir/news/98092015395/اصلی-ترین...
6996,6997,**به گزارش ایسنا،** ​​​انسولین هورمونی است که ...,https://www.isna.ir/news/98092015327/انواع-دیا...
6997,6998,فاطمه احمدی - موزه‌دار - در گفت‌وگو با ایسنا، ...,https://www.isna.ir/news/98092216157/پنجمین-مو...
6998,6999,**به گزارش ایسنا،** سیگار کشیدن، علت عمده‌ برو...,https://www.isna.ir/news/98092015395/اصلی-ترین...


In [8]:
mokasar = mokasar.set_index('jam').T.to_dict('dict')

In [9]:
suffix

['ها', 'تر', 'ترین', 'گان', 'ان', 'ات', 'ی', 'ای', 'یی']

In [10]:
df = df.dropna()

In [11]:
def tokenize(text):
  tokens = set([word for word in text.split()])
  return tokens

In [12]:
def delete_suffix(word, suffix,u=1):
  word_end = word.index(suffix[0])
  if u and '\u200c' in word and word.index('\u200c') == word_end - 1 and word.endswith(suffix):
    word = word[:word_end-1]
  elif not u and suffix in word and word.replace(suffix,'') in all_words and word.endswith(suffix):
    if suffix == 'گان':
      word = word.replace(suffix,'ه')
    else:
      word = word.replace(suffix,'')
  return word

In [13]:
def delete_punctuations(word):
  for punc in punctuations:
    if punc in word:
      word = word.replace(punc," ")
  return word

In [14]:
def delete_none_words(word):
    persian_pattern =  "^[\u0643\u0623\u0629\u0624\u0621\u0626\u0622\u0627\u0628\u067E\u062A-\u062C\u0686\u062D-\u0632\u0698\u0633-\u063A\u0641\u0642\u06A9\u06AF\u0644-\u0648\u06CC]+$"
    if re.match(persian_pattern,word):
        return word
    return ""

In [15]:
def normalize_letters(word):
    for i in len(ar):
        if ar[i] in word:
            word.replace(ar[i],pr[i])

In [16]:
def normalize_mokasar(word):
    if word in mokasar.keys():
        word = mokasar[word]['mofrad']
    return word

In [17]:
def normalize_verb(verb):
    for v in verbs:
        if verb in v:
            for p in verb_pre:
                if verb.startswith(p):
                    verb = verb.replace(p,'')
            for p in verb_post:
                if verb.endswith(p):
                    verb = verb.replace(p,'')
    return verb

In [18]:
text_list = df.content.to_list()
all_words = tokenize(" ".join(text_list))
# for i in range(len(all_words)):
#     all_words[i] = delete_none_words(all_words[i])
all_words = [delete_none_words(a_w) for a_w in all_words if delete_none_words(a_w)]

In [19]:
len(all_words)

27686

In [20]:
words_freq = dict(Counter(all_words))

In [21]:
len(words_freq)

27686

In [22]:
from tqdm import tqdm

In [23]:
def normalize_words(text_list):
  words_dict = {}
  for i in tqdm(range(len(text_list))):
    temp_words = text_list[i].split()
    temp_words = [w.strip() for w in temp_words if w.strip() not in stop_words]
    
    # print
    # for word in text_list[i].split():
    #   if word in stop_words:
    #     word = ""
    x = temp_words
    for j in range(len(temp_words)):
      temp_words[j] = delete_punctuations(temp_words[j])
      temp_words[j] = delete_none_words(temp_words[j])
      temp_words[j] = normalize_verb(temp_words[j])
      for s in suffix_u:
        if s in temp_words[j]:
          temp_words[j] = delete_suffix(temp_words[j],s)
          break
      for s in suffix:
        if s in temp_words[j]:
          temp_words[j] = delete_suffix(temp_words[j],s,u=0)
          break
    temp_words = [w for w in temp_words if w.strip() not in stop_words]

    words = list(map(str.strip, temp_words))
    for j in range(len(words)):
      if words[j] not in words_dict.keys():
        words_dict[words[j]] = {}
      if i not in words_dict[words[j]].keys():
        words_dict[words[j]][i] = 1
      else:
        words_dict[words[j]][i] += 1
  return words_dict

In [24]:
indexed_words = normalize_words(text_list)

100%|██████████| 6996/6996 [07:33<00:00, 15.42it/s]


In [25]:
del indexed_words['']

In [26]:
len(indexed_words)

23531

In [27]:
with open('indexed_words.pickle', 'wb') as handle:
    pickle.dump(indexed_words, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('indexed_words.pickle', 'rb') as handle:
    indexed_words = pickle.load(handle)

In [143]:
query = 'واکسیناسیون بیماری کرونا'

In [144]:
query = normalize_words([query])

100%|██████████| 1/1 [00:00<00:00, 128.15it/s]


In [145]:
query

{'واکسیناسیون': {0: 1}, 'بیماری': {0: 1}, 'کرونا': {0: 1}}

In [146]:
match = []
for q in query.keys():
    for iw in indexed_words.keys():
        if q == iw:
            print(iw)
            match.append(list(indexed_words[iw].keys()))
            print(len(match))
#     return match

واکسیناسیون
1
بیماری
2
کرونا
3


In [147]:
len(match)

3

In [148]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [149]:
mm = match[0]
for m in match:
    mm = intersection(mm,m)

In [150]:
mm

[5589,
 5681,
 5960,
 5999,
 6001,
 6011,
 6018,
 6034,
 6041,
 6123,
 6226,
 6269,
 6397,
 6404,
 6424,
 6431,
 6440,
 6502,
 6504,
 6505,
 6509]

In [191]:
df[df.index.isin(mm)]

NameError: name 'mm' is not defined

# فاز دو

In [ ]:
def normalize_words(text_list):
  words_dict = {}
  for i in tqdm(range(len(text_list))):
    temp_words = text_list[i].split()
    temp_words = [w.strip() for w in temp_words if w.strip() not in stop_words]
    
    # print
    # for word in text_list[i].split():
    #   if word in stop_words:
    #     word = ""
    x = temp_words
    for j in range(len(temp_words)):
      temp_words[j] = delete_punctuations(temp_words[j])
      temp_words[j] = delete_none_words(temp_words[j])
      temp_words[j] = normalize_verb(temp_words[j])
      for s in suffix_u:
        if s in temp_words[j]:
          temp_words[j] = delete_suffix(temp_words[j],s)
          break
      for s in suffix:
        if s in temp_words[j]:
          temp_words[j] = delete_suffix(temp_words[j],s,u=0)
          break
    temp_words = [w for w in temp_words if w.strip() not in stop_words]

    words = list(map(str.strip, temp_words))
    for j in range(len(words)):
      if words[j] not in words_dict.keys():
        words_dict[words[j]] = {}
      if i not in words_dict[words[j]].keys():
        words_dict[words[j]][i] = 1
      else:
        words_dict[words[j]][i] += 1
  return words_dict

In [79]:
dfs = {}
for word in indexed_words.keys():
    dfs[word] = len(indexed_words[word])

In [169]:
query = 'واکسیناسیون بیماری کرونا در جهان'

In [170]:
query = normalize_words([query])

100%|██████████| 1/1 [00:00<00:00, 209.52it/s]


In [171]:
query

{'واکسیناسیون': {0: 1}, 'بیماری': {0: 1}, 'کرونا': {0: 1}, 'جهان': {0: 1}}

In [172]:
match = []
for q in query.keys():
    for iw in indexed_words.keys():
        if q == iw:
            print(iw)
            match.append(list(indexed_words[iw].keys()))
            print(len(match))
#     return match

واکسیناسیون
1
بیماری
2
کرونا
3
جهان
4


In [173]:
len(match)

4

In [174]:
import math

In [175]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [176]:
mm = match[0]
for m in match:
    mm = intersection(mm,m)

In [177]:
def compute_distance(query, words):
    N = len(words)
    length = [0]*N
    scores = [0]*N
    for term in tqdm(query.keys()):
        print(term)
        print(dfs[term])
        idf = math.log(N/dfs[term])
        w_tq = (1+math.log(query[term][0])) * idf
        for docid in words[term].keys():
            w_td = (math.log(words[term][docid]) + 1)
            scores[docid]+= w_td * w_tq
            length[docid]+= (w_td**2)
    for docid in range(N):
        if scores[docid]:
            scores[docid] /= (length[docid]**0.5)
    return scores

In [178]:
scores = compute_distance(query,indexed_words)

100%|██████████| 4/4 [00:00<00:00, 847.42it/s]

واکسیناسیون
55
بیماری
1137
کرونا
1359
جهان
661


In [179]:
import numpy as np

In [180]:
# sorted_scores = np.argsort(scores)

In [195]:
def get_top_k(k, scores):
    sorted_scores = np.argsort(scores)
    a = sorted_scores[-1*k:]
    top_k = a[::-1]
    return top_k

In [196]:
top_10 = get_top_k(10,scores)

In [197]:
top_10

array([6474, 6727, 6724, 6721, 6226, 5960, 6404, 5681, 6509, 3207])

In [198]:
# for d in top_k:
print(df.loc[top_10[0]])

id                                                      6475
content    ابراهیم رضایی در ارائه گزارش به خبرنگاران به م...
url        https://www.isna.ir/news/99121209285/استان-بوش...
Name: 6474, dtype: object


# Champion List

In [199]:
def make_champions(r, indexed_words):
    champion_list = {}
    for term in indexed_words:
        doc_dict = indexed_words[term].copy()
        champions = dict(sorted(doc_dict.items(), key=lambda x: x[1], reverse=True)[:r])
        champion_list[term] = champions
    return champion_list

In [200]:
champion_list = make_champions(5,indexed_words)
champion_sims = compute_distance(query,champion_list)

100%|██████████| 4/4 [00:00<00:00, 2358.67it/s]

واکسیناسیون
55
بیماری
1137
کرونا
1359
جهان
661


In [201]:
top_10 = get_top_k(10,champion_sims)

In [202]:
top_10

array([6407, 5960, 6454, 6269, 6440, 6123, 1317,  977, 1623,   32])

In [203]:
print(df.loc[top_10[0]])

id                                                      6408
content    سیدعلی آقازاده در گفت‌وگو با ایسنا، اظهار کرد:...
url        https://www.isna.ir/news/99112014708/واکسن-کرو...
Name: 6407, dtype: object


In [38]:
!pip3 install xlrd

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# فاز سه

In [45]:
news_11k = pd.read_csv('IR00_3_11k News.csv')
news_17k = pd.read_csv('IR00_3_17k News.csv')
news_20k = pd.read_csv('IR00_3_20k News.csv')

In [46]:
news_11k

,id,content,topic,url
0,1,به گزارش ایسنا، پس از استعفا علی نکویی رئیس شا...,sport,https://www.isna.ir/news/99010100077/حواشی-در-...
1,2,به گزارش ایسنا، امیر محسنی با بیان اینکه این ل...,sport,https://www.isna.ir/news/98122922468/ثبت-نام-ب...
2,3,آرش فرهادیان در گفت و گو با ایسنا، درباره آخری...,sport,https://www.isna.ir/news/99010200541/فرهادیان-...
3,4,به گزارش ایسنا، فدراسیون بین المللی شنا قانونی...,sport,https://www.isna.ir/news/99010200528/فناوری-وی...
4,5,به گزارش ایسنا، فدراسیون جهانی صعودهای ورزشی (...,sport,https://www.isna.ir/news/99010200510/تعویق-زما...
...,...,...,...,...
11432,11433,به گزارش ایسنا، کمپین کرونا فایترز (Corona Fig...,culture,https://www.isna.ir/news/98122922416/جشنواره-ه...
11433,11434,عباسی انارکی در گفت‌وگویی با ایسنا در پی برخی ...,culture,https://www.isna.ir/news/98122922382/توضیحات-م...
11434,11435,**به گزارش ایسنا** ، دکتر محمد منصورنژاد در یا...,culture,https://www.isna.ir/news/98122922381/بیماری-ها...
11435,11436,ایرج راد، رئیس هیئت مدیره خانه تئاتر که از طری...,culture,https://www.isna.ir/news/98122922326/بهبودی-به...


In [48]:
news_17k

,id,content,topic,url
0,1,یوسف مرادلو در گفت‌وگو با خبرنگار اقتصادی خبرگ...,economy,https://www.farsnews.ir/news/14000205000595/را...
1,2,به گزارش خبرنگار اقتصادی خبرگزاری فارس، حدود س...,economy,https://www.farsnews.ir/news/14000205000985/بر...
2,3,علی اوسط هاشمی در گفت‌وگو با خبرنگار اقتصادی خ...,economy,https://www.farsnews.ir/news/14000205000818/تد...
3,4,به گزارش خبرنگار اقتصادی خبرگزاری فارس، موضوع ...,economy,https://www.farsnews.ir/news/14000204000416/کد...
4,5,به گز‌ارش خبرنگار اقتصادی خبرگز‌اری فارس روز چ...,economy,https://www.farsnews.ir/news/14000205000641/پر...
...,...,...,...,...
17959,17960,به گزارش خبرنگار قرآن و فعالیت‌های دینی خبرگزا...,culture,https://www.farsnews.ir/news/13990423000200/بر...
17960,17961,به گزارش خبرنگار کتاب و ادبیات خبرگزاری فارس، ...,culture,https://www.farsnews.ir/news/13990423000095/بر...
17961,17962,**خبرگزاری فارس ـ حوزه مسجد و هیأت ـ امیرحسین ...,culture,https://www.farsnews.ir/news/13990421000709/اح...
17962,17963,به گزارش خبرنگار حوزه مسجد و هیأت خبرگزاری فار...,culture,https://www.farsnews.ir/news/13990422000816/مس...


In [49]:
news_20k

,id,content,topic,url
0,1,به گزارش خبرگزاری فارس، به بهانه انتقال یولیان...,sports,https://www.farsnews.ir/news/14000207000773/گر...
1,2,به گزارش خبرگزاری فارس، ژردان شقیری قصد دارد ب...,sports,https://www.farsnews.ir/news/14000207000576/شق...
2,3,به گزارش خبرگزاری فارس، اورتونی ها برای فصل بع...,sports,https://www.farsnews.ir/news/14000207000599/او...
3,4,به گزارش خبرگزاری فارس، نهمین مرحله اردوی تیم ...,sports,https://www.farsnews.ir/news/14000207000678/تی...
4,5,به گزارش خبرگزاری فارس، نیکلاس زوله حدود 3 هفت...,sports,https://www.farsnews.ir/news/14000207000483/زو...
...,...,...,...,...
20655,20656,به گزارش حوزه پارلمانی خبرگزاری فارس، [محمدباق...,politics,https://www.farsnews.ir/news/13990612000494/پی...
20656,20657,به گزارش خبرنگار تشکل‌های دانشگاهی خبرگزاری فا...,politics,https://www.farsnews.ir/news/13990612000397/اع...
20657,20658,به گزارش خبرنگار حوزه دولت خبرگزاری فارس، رضا ...,politics,https://www.farsnews.ir/news/13990612000425/اف...
20658,20659,به گزارش خبرنگار دولت خبرگزاری فارس، محسن حاجی...,politics,https://www.farsnews.ir/news/13990612000435/وز...


In [121]:
with open('indexed_words.pickle', 'rb') as handle:
    indexed_words = pickle.load(handle)

In [122]:
len(indexed_words)

23531

In [123]:
def get_length(doc):
    length = 0
    for term in doc.keys():
        length += doc[term]**2
    length = length**0.5
    return length

In [124]:
def make_doc_vectors(words):
    documents = {}
    lengths = [0]*len(documents)
    for word in words.keys():
        for doc in words[word].keys():
            if doc not in documents.keys():
                documents[doc] = {}
            documents[doc][word] = words[word][doc]
    return documents

In [125]:
documents = make_doc_vectors(indexed_words)

In [126]:
import random

In [242]:
def choose_centers(k,documents,doc_center):
    diff_docs = []
    for i in doc_center:
        if i not in diff_docs:
            diff_docs.append(i)
    print(diff_docs)
    centers = [0]*k
    counter = [0]*k
    for doc in documents.keys():
        center_index = doc_center[doc]
        centers[center_index] = {}
        counter[center_index] += 1
        for term in documents[doc]:
            if term in centers[center_index]:
                centers[center_index][term]+=documents[doc][term]
            else:
                centers[center_index][term] = documents[doc][term]
#     print(centers)
    for center in centers:
        if not center:
            print(centers.index(center))
#     for center in range(len(centers)):
#         if not centers[center]:
#             continue
#         for word in centers[center].keys():
#             centers[center][word]/=counter[center]
    return centers

In [243]:
def k_means(k,documents):
    centers = []
    doc_centers = [0]*len(documents)
    for i in range(k):
        centers.append(documents[random.randint(0,len(documents))])
    for i in tqdm(range(20)):
        for doc in documents.keys():
            max_sim = -1*math.inf
            for center in centers:
                if not center:
                    continue
                sim = 0
                for term in documents[doc].keys():
# #                     if term == 'center':
#                         continue
                    if term in center.keys():
#                         print(term)
#                         print('1',math.log(documents[doc][term]))
# #                         print('2',math.log(len(documents)/dfs[term]))
#                         print('3',math.log(center[term]))
                        sim+= (math.log(documents[doc][term])+1) * math.log(len(documents)/dfs[term]) * (math.log(center[term])+1) 
#                         print(sim)
                sim/= get_length(center)
                if sim > max_sim:
                    max_sim = sim
                    max_center = center
            doc_centers[doc] = centers.index(max_center)
        centers = choose_centers(k,documents,doc_centers)
    return doc_centers, centers

In [244]:
new_docs, centers = k_means(5,documents.copy())

  5%|▌         | 1/20 [00:02<00:47,  2.49s/it]

[0, 2, 1, 4, 3]
[3, 4, 2, 0, 1]


 10%|█         | 2/20 [00:03<00:39,  2.17s/it]

[3, 4, 2, 0, 1]


 15%|█▌        | 3/20 [00:05<00:33,  1.96s/it]

...

 90%|█████████ | 18/20 [00:28<00:03,  1.54s/it]

[3, 4, 2, 0, 1]
[3, 4, 2, 0, 1]


 95%|█████████▌| 19/20 [00:29<00:01,  1.51s/it]

[3, 4, 2, 0, 1]


100%|██████████| 20/20 [00:31<00:00,  1.55s/it]


centers

In [245]:
query = 'جام جهانی فوتبال'

In [246]:
def get_result(k,query,centers, doc_centers, documents):
    query_words = normalize_words([query])
    q_doc = make_doc_vectors(query_words)[0]
#     print(q_doc)
    max_sim = -1*math.inf
    for center in centers:
        sim = 0
        for term in q_doc.keys():
            if term in center:
                print('گوه')
                sim += (math.log(center[term])+1)*(math.log(q_doc[term])+1)*math.log(len(documents)/dfs[term])
        if sim>max_sim:
            print(sim,max_sim)
            max_sim = sim
            max_center = center
    center_index = centers.index(max_center)
    print(center_index)
    doc_similarities = {}
    for doc in documents.keys():
        if doc_centers[doc]==center_index:
            sim = 0
            for term in q_doc.keys():
#                 print(doc)
                if term in documents[doc].keys():
                    sim+= (math.log(documents[doc][term])+1)*(math.log(q_doc[term])+1)*math.log(len(documents)/dfs[term])
            doc_similarities[doc] = sim
    sorted_sims = [k for k, v in sorted(doc_similarities.items(), key=lambda item: item[1], reverse=True)]
    return sorted_sims[:k]

In [247]:
centers

[{'انتهای': 1,
  'پیام': 1,
  'گفت': 1,
  'گو': 1,
  'ش': 1,
  'اعضا': 1,
  'تصویب': 1,
  'مورد': 1,
  'برخی': 2,
  'ایسنا': 1,
  'بررس': 2,
  'کمیسیون': 3,
  'جلسه': 3,
  'قرار': 1,
  'حمایت': 1,
  'امروز': 3,
  'شورا': 1,
  'سو': 1,
  'خصوص': 1,
  'فکر': 1,
  'مواد': 3,
  'فرهنگ': 2,
  'اسلام': 1,
  'مجلس': 2,
  'فاطمه': 1,
  'لایحه': 3,
  'مالکیت': 1,
  'محول': 1,
  'سیده': 1,
  'ذوالقدر': 1},
 {'انتهای': 1,
  'پیام': 1,
  'میلیون': 1,
  'هزار': 1,
  'بهبود': 1,
  'هدف': 1,
  'حجم': 2,
  'منطقه': 1,
  'احداث': 1,
  'طول': 1,
  'م': 1,
  'سالیانه': 2,
  'آبیاری': 1,
  'افتتاح': 1,
  'تاج': 1,
  'سد': 1,
  'رودخانه': 1,
  'لنگرود': 1,
  'هکتار': 1,
  'مخزن': 1,
  'مترمکعب': 1,
  'تنظیمی': 1,
  'لاستیکی': 1,
  'گالشکلام': 1,
  'شالیزارهای': 1,
  'شلمانرود': 1},
 {'انتهای': 1,
  'پیام': 1,
  'برگزار': 1,
  'نام': 1,
  'توجه': 1,
  'اظهار': 1,
  'عدم': 1,
  'ایسنا': 1,
  'نسبت': 1,
  'قرار': 1,
  'گفته': 1,
  'اجرا': 1,
  'همچون': 1,
  'جنوب': 1,
  'قاسم': 1,
  'اختصاص': 1,
  'تحقیق': 1,

In [236]:
query = 'افتتاح آبیاری باغ'

In [237]:
results = get_result(10,query,centers,new_docs,documents)

100%|██████████| 1/1 [00:00<00:00, 327.76it/s]

گوه
گوه
-54.64889577880403 -inf
0 -54.64889577880403
گوه
1


In [238]:
results

[3727, 268, 1185, 2418, 3365, 3916, 4313, 5230, 5647, 5653]

In [239]:
df[df.index.isin(results)]

,id,content,url
268,269,به گزارش ایسنا، زنیت توانست با برتری ۴ بر ۲ بر...,https://www.isna.ir/news/99041611866/آزمون-قهر...
1185,1186,حسین نقیبی در گفتگو با ایسنا، درباره برگزاری د...,https://www.isna.ir/news/98040301555/حسن-یزدان...
2418,2419,به گزارش ایسنا، اظهارات یکی از مهمانان برنامه ...,https://www.isna.ir/news/99110906701/زاویه-دول...
3365,3366,**به گزارش ایسنا،** در مراسمی با حضور سورنا ست...,https://www.isna.ir/news/99022215951/ساخت-ربات...
3727,3728,به گزارش ایسنا، در طول سال‌های گذشته با توجه ب...,https://www.isna.ir/news/99060504758/چرا-کارت-...
3916,3917,حمید بردستانی با اشاره به اهمیت بالای اجرای طر...,https://www.isna.ir/news/99081308414/اجرای-طرح...
4313,4314,محمود نجفی عرب در گفت‌گو با ایسنا، با اشاره به...,https://www.isna.ir/news/98011604737/آیا-معافی...
5230,5231,علی لاریجانی عصر امروز 12 دی ماه در آئین تجلیل...,https://www.isna.ir/news/98101209165/جواب-توئی...
5647,5648,**به گزارش ایسنا،** انجمن علمی آموزش بهداشت و ...,https://www.isna.ir/news/99031308227/چند-راهکا...
5653,5654,**به گزارش ایسنا،** انجمن علمی آموزش بهداشت و ...,https://www.isna.ir/news/99031308227/چند-راهکا...


In [240]:
query = 'حقوق معلمان چند تومان است'

In [241]:
results = get_result(10,query,centers,new_docs,documents)

100%|██████████| 1/1 [00:00<00:00, 176.70it/s]

0 -inf
گوه
0


In [227]:
results

[296, 6445, 3284, 3570, 5869, 6114, 4551, 5310, 4766, 2492]

In [228]:
df[df.index.isin(results)]

,id,content,url
296,297,به گزارش ایسنا، در جریان برگزاری مجمع عمومی سا...,https://www.isna.ir/news/99042518993/گزارش-مال...
2492,2493,به گزارش ایسنا، بابک نگاهداری گفت: ایران اسلام...,https://www.isna.ir/news/99122519381/سامانه-ار...
3284,3285,محمد خدابخشی در گفت‌وگو با ایسنا اظهار کرد: جل...,https://www.isna.ir/news/99020201675/بررسی-تفر...
3570,3571,به گزارش ایسنا، به گفته واردکنندگانی که در صف ...,https://www.isna.ir/news/99042619732/صادرکنندگ...
4551,4552,به گزارش ایسنا، حمدالله کریمی روز چهارشنبه در ...,https://www.isna.ir/news/98030803635/جاده-ای-ک...
4766,4767,به گزارش ایسنا، جدیدترین آمار منتشر شده از سوی...,https://www.isna.ir/news/98051005054/رشد-تولید...
5310,5311,به گزارش ایسنا، امروز (سه‌شنبه ۱۵ بهمن‌ماه) فر...,https://www.isna.ir/news/98111511025/رشد-۵۰-در...
5869,5870,به گزارش ایسنا، کادر درمانی و بیماران بیمارستا...,https://www.isna.ir/news/99051712874/اجرای-موس...
6114,6115,سعید کشمیری با بیان اینکه تعداد بیماران بستری ...,https://www.isna.ir/news/99072518690/۸-شهرستان...
6445,6446,**به گزارش ایسنا،** دکتر سیماسادات لاری گفت: ا...,https://www.isna.ir/news/99120301765/۷۴-فوتی-و...


In [54]:
words_11k = normalize_words(text_list11)

 55%|█████▌    | 6302/11437 [07:55<06:27, 13.25it/s]


KeyboardInterrupt: 

In [53]:
text_list11 = news_11k.content.to_list()